In [38]:
import pandas as pd

In [39]:
stats = pd.read_csv("player_stats.csv")
del stats["Unnamed: 0"]
stats

,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,...,Share,Tm,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,A.C. Green,30,PF,82,55,34.5,5.7,11.3,0.502,0.1,...,0.000,PHO,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
1,Cedric Ceballos,24,SF,53,43,30.2,8.0,15.0,0.535,0.0,...,0.000,PHO,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
2,Charles Barkley,30,PF,65,65,35.4,8.0,16.1,0.495,0.7,...,0.005,PHO,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
3,Dan Majerle,28,SG,80,76,40.1,6.0,14.2,0.418,2.4,...,0.000,PHO,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
4,Danny Ainge,34,SG,68,1,22.9,3.3,7.9,0.417,1.2,...,0.000,PHO,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,Jordan McLaughlin,23,PG,30,2,19.7,2.9,5.9,0.489,1.0,...,0.000,MIN,Minnesota Timberwolves,19,45,0.297,22.5,113.3,117.5,-4.02
12405,Josh Okogie,21,SG,62,28,25.0,2.7,6.4,0.427,0.7,...,0.000,MIN,Minnesota Timberwolves,19,45,0.297,22.5,113.3,117.5,-4.02
12406,Karl-Anthony Towns,24,C,35,35,33.9,9.0,17.8,0.508,3.3,...,0.000,MIN,Minnesota Timberwolves,19,45,0.297,22.5,113.3,117.5,-4.02
12407,Kelan Martin,24,SF,31,4,16.0,2.3,5.8,0.392,0.8,...,0.000,MIN,Minnesota Timberwolves,19,45,0.297,22.5,113.3,117.5,-4.02


In [40]:
# Finding null values in the columns
pd.isnull(stats).sum()

Player         0
Age            0
Pos            0
G              0
GS             0
MP             0
FG             0
FGA            0
FG%           55
3P             0
3PA            0
3P%         1711
2P             0
2PA            0
2P%           98
eFG%          55
FT             0
FTA            0
FT%          484
ORB            0
DRB            0
TRB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
Awards     10795
Year           0
Pts Won        0
Pts Max        0
Share          0
Tm             0
Team           0
W              0
L              0
W/L%           0
GB             0
PS/G           0
PA/G           0
SRS            0
dtype: int64

In [41]:
# Percentages were null because certain players hadn't attempted those shots

# Filling those cases with 0
stats = stats.fillna(0)

In [42]:
# Training the algorithm
# stats.columns
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
        'W', 'L', 'W/L%', 'GB',
       'PS/G', 'PA/G', 'SRS']

In [43]:
train = stats[stats["Year"] < 2023]
test = stats[stats["Year"] == 2023]
test2 = stats[stats["Year"] == 2022]

In [44]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [45]:
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [46]:
predictions = reg.predict(test[predictors])
predictions2 = reg.predict(test2[predictors])
# predictions

In [47]:
predictions = pd.DataFrame(predictions, columns=["23Score"], index=test.index)
predictions2 = pd.DataFrame(predictions2, columns=["22Score"], index=test2.index)
# predictions

In [48]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
combination2 = pd.concat([test2[["Player", "Share"]], predictions2], axis=1)
combination

,Player,Share,23Score
116,A.J. Green,0.000,0.003226
117,Bobby Portis,0.000,0.026498
118,Brook Lopez,0.000,0.039110
119,Giannis Antetokounmpo,0.606,0.223999
120,Grayson Allen,0.000,-0.009855
...,...,...,...
12214,Mitchell Robinson,0.000,-0.015891
12215,Obi Toppin,0.000,0.002912
12216,Quentin Grimes,0.000,-0.014665
12217,RJ Barrett,0.000,0.020850


In [49]:
combination_sorted = combination.sort_values(by="23Score", ascending=False)
combination_sorted.head(50)

,Player,Share,23Score
119,Giannis Antetokounmpo,0.606,0.223999
11938,Joel Embiid,0.915,0.196260
11470,Luka Dončić,0.000,0.191829
491,Nikola Jokić,0.000,0.159644
928,Shai Gilgeous-Alexander,0.046,0.147268
6231,Damian Lillard,0.000,0.135751
2132,Jayson Tatum,0.280,0.133313
6577,Anthony Davis,0.000,0.132656
6581,LeBron James,0.000,0.126562
9956,Ja Morant,0.001,0.125090


In [50]:
combination_sorted2 = combination2.sort_values(by="22Score", ascending=False)
combination_sorted2.head(50)

,Player,Share,22Score
9405,Giannis Antetokounmpo,0.595,0.213778
608,Joel Embiid,0.706,0.187846
475,Nikola Jokić,0.000,0.172505
9237,Luka Dončić,0.000,0.151256
6568,LeBron James,0.001,0.146221
9039,Kevin Durant,0.001,0.132530
9936,Ja Morant,0.010,0.116031
9565,Trae Young,0.000,0.105539
6559,Anthony Davis,0.000,0.101290
3068,DeMar DeRozan,0.001,0.096749


In [ ]:
# Creating a dataframe that holds the two seasons scores
two_szns = pd.merge(combination_sorted, combination_sorted2, on="Player", suffixes=("_1", "_2"))
two_szns = two_szns[["Player", "23Score", "22Score"]]
two_szns = two_szns.fillna(0)

In [58]:
two_szns["Improve"] = two_szns["23Score"] - two_szns["22Score"]
two_szns = two_szns.sort_values(by="Improve", ascending=False)

In [59]:
# Displaying the top 50 players who improved the most from the 2022 to 2023 seasons
two_szns.head(50)

,Player,23Score,22Score,Improve
5,Damian Lillard,0.135751,0.060939,0.074812
21,Lauri Markkanen,0.075236,0.008730,0.066506
4,Shai Gilgeous-Alexander,0.147268,0.086372,0.060896
42,Shaquille Harrison,0.039145,-0.018849,0.057994
66,Michael Porter Jr.,0.022407,-0.027339,0.049746
20,Jalen Brunson,0.075373,0.027218,0.048155
28,Nic Claxton,0.056406,0.008982,0.047424
50,Kris Dunn,0.033197,-0.013383,0.046580
23,Myles Turner,0.067566,0.021100,0.046466
6,Jayson Tatum,0.133313,0.088273,0.045040
